# Sequence Modeling: Recurrent and Recursive Nets
**Goodfellow Chapter 10**

**Recurrent neural networks**, or RNNs, are a family of neural networks for processing sequential data. Similar to how a convolutional network is designed to process a grid of values $\textbf X,$ an RNN is designed to process a sequence of values $\textbf{x}^{1}, . . ., \textbf{x}^\tau$. RNNs can process longer sequences than would be practical with other architectures, and can typically process sequences of variable length.

The key idea needed to move from feedforward to recurrent networks is parameter sharing. RNNs need to share parameters across different parts of the model in order to generalize to different sequence lengths. This is also important because the same piece of information can often appear in different positions within a sequence. 

Suppose we want to train a model to extract years from text. The sentence "I went to Nepal in 2009" contains the same information if it is rewritten as "In 2009, I went to Nepal." A traditional feedforward network, trained on sentences of a fixed length with a different parameter for each input feature, would need to learn what a year looked like in each position of the sentence. This requires a lot of redundant learning. An RNN, alternatively, shares the same weights across several time steps, eliminating the need to re-learn the rules of language for each position in a sequence.

An RNN shares parameters by passing a function of each step's output to the next position in the sequence. This way, the model always has a sense of context learned from previous steps' parameters. 

## Unfolding Computational Graphs

Recall that a computational graph is a way to formalize the structure of a set of computations, mapping inputs and parameters to outputs and loss values. The feature of the computational graph that makes a network _recurrent_ is a connection between the nodes of one sequence's graph and another's. This requires **unfolding** the graph to represent  its multiple time steps to represent a chain of events. 

For example, consider the classical form of a dynamical system:

$$ \textbf{s}^{(t)} = f(\textbf{s}^{(t-1)};\mathbf{\theta}), $$

where $\textbf{s}^{(t)}$ is called the state of the system. This equation is recurrent because at each step, it calls upon the same function at a previous state.

For a finite number of steps $\tau,$ we can unfold this graph by applying the function $\tau - 1$ times. For example, with $\tau = 3,$ the unfolded version of the above graph becomes:


$$
\textbf{s}^{(3)} = f(\textbf{s}^{(2)};\mathbf{\theta}) \\
= f(f(\textbf{s}^{(1)};\mathbf{\theta});\mathbf{\theta})
$$

The unfolded function is no longer recurrent, and can now be represented as an acyclic graph.

A typical RNN will use the following equation, differing only in its use of the input data $\mathbf{x}^{(t)},$ with $\textbf{h}$ representing the model state:

$$ \textbf{h}^{(t)} = f(\textbf{h}^{t - 1}, \textbf{x}^{t};\mathbf{\theta}) $$

When an RNN is tasked with predicting the future given the past items in a sequence, it learns to use $\textbf{h}^{(t)}$ as a lossy summary of the task-relevant aspects of the past sequence of inputs up to $t.$ This summary is in general lossy, since it maps an arbitrary length sequence to a fixed length vector $\textbf{h}^{(t)}.$ This will typically involve the state "forgetting" pieces of the past sequence that it has deemed irrelevant. 

The unfolded recurrence after $t$ steps can be represented as:

$$ 
\textbf{h}^{(t)} = g^{(t)}(x^{(t)}, x^{(t-1)}, . . ., x^{(2)}, x^{(1)})) \\
= f(\textbf{h}^{(t-1)}, \textbf{x}^{(t)};\mathbf{\theta}).$$

The advantages of this unfolded representation are:

* The learned model always has the same input size regardless of sequence length
* It is possible to use the same transition function $f$ with the same parameters at every step

This means we get to learn a single model $f$ that operates on all times steps and all sequence lengths, rather than needing a separate model $g^{(t)}$ for each possible time step. 

Learning a single shared model allows generalization to sequence lengths that did not appear in the training set, and enables the model to be estimated with relatively few training examples. 

## Recurrent Neural Networks

A few design patterns for RNNs are:

* Producing an output at each time step, with recurrent connections between hidden units
* Producing an output at each time step, with a recurrent connection between the output on step t-1 and the hidden unit of step t
* RNNs with recurrent connections between hidden units, producing a single output after processing the entire sequence

A standard RNN forward propogation would be:

$$ \textbf{a}^{(t)} = \textbf b + \textbf W \textbf{h}^{t-1} + \textbf{Ux}^{(t)}, $$

$$ \textbf{h}^{(t)} = \text{tanh} \left ( \textbf{a}^{(t)} \right ), $$

$$ \textbf{o}^{(t)} = \textbf c + \textbf{Vh}^{(t)}, $$

$$ \mathbf{\hat{y}}^{(t)} = \text{softmax} \left ( \textbf{o}^{(t)} \right ),$$

where $\textbf U , \textbf V, \text{and } \textbf W$ are the input-to-hidden, hidden-to-output, and recurrent hidden-to-hidden weights respectively. The loss, then, would be the sum of the log losses for each time step. Backpropogation through an RNN involves moving back through each time step, end to beginning. 

### Teacher Forcing and Networks with Output Recurrence

Networks with output recurrence are strictly less powerful than nestworks with hidden layer recurrence. Models with this type of recurrence can be trained with **teacher forcing**. This method simply involves feeding the previous step's output as an input to a hidden layer in the next step. 

### Computing the Gradient in an RNN

[yikes, come back to this]

## Bidirectional RNNs

The RNNs discussed so far take into accound only past and present information from the sequence (x_1, ..., x_t). Many sequence processing tasks, however, require us to take into account information from both before and after the time-step of interest. Take voice recognition as an example. In order to guess the words being spoken in raw audio, we need to know the sounds happening both before and after the sounds at step t in the sequence. This is because transition probabilities between sounds (i.e. the likelihood of certain phonemes happening before and after a candidate word) carry crucial information for this task. For this reason, it would be useful to have a recurrent model that is able to look into the future for added context. 

Bidirecitonal RNNs answer this need, combining an RNN that moves forward through time, beginning at the sequence start, and a second RNN that moves backward, starting at the end of the sequence and moving toward position 0. These RNNs have two states for the forward and backward-moving networks, which effectively represent knowledge of the past and future of the sequence at each time step. Similar representations can be formed for two-dimensional data such as images, with four RNNs, one moving in each direction (up/down/left/right).

Bidirectional RNNs have shown success in speech, handwriting recognition, and bioinformatics. 

## Encoder-Decoder Sequence-to-Sequence Architecture

We often want our RNN to be able to map variable-length input sequences to variable-length output sequences. Examples of this include translation and question-answering tasks. The simplest architecture for doing this is called either an **encoder-decoder** or **sequence-to-sequence** model.

The approach is:

* First, an **encoder** or **reader** or **input** RNN processes the input sequence. The encoder emits a contex C, usually a simple function of its final hidden state
* Then, a **decoder** or **writer** or **output** RNN is conditioned on the fixed-length vector passed by the previous RNN to generate the output sequence $P(Y=(y^{(1)}, ..., y^{(n_{y})}).$

The benefit of such a model is that the two lengths, $n_{y} \text{ and } n_{x}$ do not need to be the same, where other architectures have the constraint $n_{y} = n_{x} = \tau.$ 

The two RNNs are trained jointly to maximize the average of $\text{log}P(y^{(1)}, ..., y^{(n_{y})}|x^{(1)}, ..., x^{(n_{x})})$ over all (x,y) sequences. The final state $\mathbf{h}_{n_{x}}$ of the encoder is typically used as a representation C of the input sequence that is provided as input to the decoder RNN. 

## Deep Recurrent Networks

The computations of must RNNs fall into three blocks of parameters and associated transformations:

* from input to hidden state
* from hidden state to hidden state
* from hidden state to output,

Where each of these bullet points corresponds to a single learned weight matrix. It may sometimes be beneficial, however, to use more than a single hidden layer. 

Empirical evidence shows that multilayer RNNs often outperform shallow ones. Optimization, however, is in general more difficult as networks become more deep. One way to speed up computation and training in these cases, however, is to use skip-connections, which skip hidden layers and allow the gradient to flow more freely between input and output. 

## The Challenge of Long-Term Dependencies

The challenge with learning long-term dependencies in recurrent models is that gradients propogated over many stages either vanish (most of the time) or explode (rarely.) The problem is that weights involving long-term dependencies fade away through the multiplication of many Jacobians, where short-term dependencies tend to have larger weights. Most recent RNN research involves finding ways to overcome this vanishing long-term dependency problem. 

RNNs involve the composition of the same function over multiple time steps. These compositions can result in extremely nonlinear behavior. In particular, the function composition employed by RNNs somewhat resembles matrix multiplication. We can think of the recurrence relation

$$ \mathbf{h}^{(t)} = \left ( \mathbf{W}^{t} \right )^\intercal \textbf{h}^{(0)}, $$

and if $\textbf W$ admits an eigendecomposition of the form

$$ \textbf W = \textbf{QAQ}^\intercal, $$

with orthogonal $\textbf Q,$ the recurrence may be simplified further to 

$$ \textbf{h}^{(t)} = \textbf{Q}^\intercal \textbf{A}^{(t)} \textbf{h}^{(0)}. $$

The eigenvalues are raised to the power of t, causing eigenvalues with magnitude less than one to decay to zero, and eigenvalues with magnitude greater than one to explode. Any component of $\textbf{h}^{(0)}$ that is not aligned with the largest eigenvector will inevitably be discarded. 

This problem is specific to recurrent networks. Imagine multiplying a weight $w$ by itself any number of times. The weight will inevitably either decay to zero or explode toward infinity. 

## Leaky Units and Other Strategies for Multiple Time Scales

One way to deal with long-term dependencies is to design a model that operates at multiple time scales, with some parts operating at small time scales and others on longer time scales. These strategies include using skip connections, leaky units, and removing some of the connections used in fine-grained time scales. 

### Adding Skip Connections through Time

One way to obtain a coarse time scale is to add direct connections between the distant past and the present, skipping one or many intermediate time steps. 

Looking back to the problem of gradients disappearing as a function of the number of time steps $t$, using skip connections to represent a time delay of $d$ steps delays the diminishing of the gradient to a more reasonable function of $\frac{t}{d}.$

Since there are both delayed and single step connections, the gradient stil lmay explode exponentially in $t$. This allows the learning algorithm to capture longer dependencies. 

### Leaky Units and a Spectrum of Different Time Scales

Another way to obtain paths on which the product of dreivatives is close to one is to have units with linear self-connections and a weight near one on these connections.

When we accumulate a running average $\mu^{(t)}$ of some value $v^{(t)}$ by applying the update $\mu^{(t)} = \alpha \mu^{(t-1)} + (1 - \alpha)v^{(t)},$ the $\alpha$ parameter is an example of a linear self-connection from $\mu^{(t-1)}$ to $\mu^{(t)}$. WHen $\alpha$ is close to one, the running average remembers information about the past for a long time, and when $\alpha$ is near zero, it discards most of what it knows about the past. Hidden units with linear self-connections can behave similarly to such running averages. These hidden units are called **leaky units**. 

Skip connections through $d$ time steps ensure that a unit can learn from the hidden state $d$ steps earlier. The use of a linear self-connection with a weight near one is different because it allows the effect to be adapted more smoothly and flexibly by adjusting th weight $\alpha$ rather than by adjusting the integer-valued skip length $d$. 

The time constants used by leaky units can either be drawn from a distribution and fixed in place or learned as parameters in the model. 

### Removing Connections

Another approach to the long-term dependency problem is removing short-term connections and replacing them with long-term ones. This is different from a skip-connection strategy because it is not adding any additional connections, but rather repurposing existing ones to be used in longer time scales. 

## The Long Short-Term Memory (LSTM) and Other Gated RNNs

The most effective RNNs today are **gated RNNs**, or RNNs that learn weights that can change with each time step. Among these models are the LSTM and those based on the gated recurrent unit (GRU).

### LSTM

The main idea of the LSTM is that it contains self-loops that produce paths where the gradient can flow for long durations. The weight of this self-loop is conditioned on the context (i.e. the loop is gated). This model has shown success in handwriting recognition, speech recognition, machine translation, image captioning, and other tasks.  

Instead of having hidden units solely apply an element-wise nonlinearity to the affine transformation of inputs and recurrent units, the LSTM cell has an internal recurrence (self-loop) in addition to the standard RNN outer recurrence. Each cell has the same input and output as the standard RNN cell, but also has additional within-cell parameters and a system of gating units that controls the flow of information. 

The most important gate is the state unit $s_{i}^{(t)},$ which has a linear self-loop similar to a leaky unit. Here, however, the self-loop weight (i.e. the time constant) is controlled by a **forget gate** unit $f_{i}^{(t)},$ for time step t and cell i, which sets this weight to a value between 0 and 1 via a sigmoid unit:

$$f_{i}^{(t)} = \sigma \left (  b_{i}^{f} + \sum_{j} U_{i,j}^f x_{j}^{(t)} + \sum_{j} W_{i,j}^f h_{j}^{(t-1)}  \right ),  $$

where $x_{j}^{(t)}$ is the current input vector, $h^{(t)}$ is the current hiden layer vector, containing the outputs of all the LSTM cells, and $b^f, U^f, W^f$ are the biases, input weights, and recurrent weights for the forget gates. 
 
The LSTM cell is updated as follows:

$$ s_{i}^{(t)} = f_{i}^{(t)}s_{i}^{(t-1)} + g_{i}^{(t)} \sigma \left ( b_{i} + \sum_{j} U_{i,j} x_{j}^{(t)} + \sum_{j} W_{i,j} h_{j}^{(t-1)}  \right ).$$

The **external input gate** unit $g_{i}^{(t)}$, is computed in a similar tay to the forget gate (with a sigmoid unit to obtain a gating value between 0 and 1), but with its own parameters:

$$ g_{i}^{(t)} = \sigma \left (  b_{i}^{g} + \sum_{j} U_{i,j}^{g}x_{j}^{(t)} + \sum_{j}W_{i,j}^{g}h_{j}^{(t-1)} \right ) .$$

The output $h_{i}^{(t)}$ of the LSTM cell can be turned off via the output gate $q_{i}^{(t)},$ which also uses a sigmoid for gating:

$$ h_{j}^{(t)} = \text{tanh} \left ( s_{i}^{(t)} \right ) q_{i}^{(t)}, $$

$$ q_{i}^{(t)} = \sigma \left (   b_{i}^{o} + \sum_{j}U_{i,j}^{o}x_{j}^{(t)} + \sum_{j}W_{i,j}^{o}h_{j}^{(t-1)}  \right ), $$

where $b^{o}, U^{o}, \text{ and } W^{o}$ are the bias, input weights, and recurrent weights. 

LSTM networks have been shown to perform better than simple recurrent architectures in learning long-term dependencies.


### Other Gated RNNs

Other RNN research has focused on gated recurrent units (GRUs) which involve less steps than the computationally expensive LSTM. The most commonly used GRU, for example, uses a single gate for both the forgetting factor and the update operation. The update equations are: 

$$ h_{i}^{(t)} = u_{i}^{(t-1)}h_{i}^{(t-1)} + (1 - u_{i}^{(t-1)}) \sigma \left ( b_{i} + \sum_{j} U_{i,j} x_{j}^{(t-1)} + \sum_{j} W_{i,j} r_{j}^{(t-1)} h_{j}^{(t-1)}  \right ), $$

where $u$ is the update gate and r is the reset gate. Their own values are defined as usual: 

$$ u_{i}^{(t)} = \sigma \left (   b_{i}^{u} + \sum_{j}U_{i,j}^{u}x_{j}^{(t)} + \sum_{j}W_{i,j}^{u}h_{j}^{(t-1)}  \right ), $$

$$ r_{i}^{(t)} = \sigma \left (   b_{i}^{r} + \sum_{j}U_{i,j}^{r}x_{j}^{(t)} + \sum_{j}W_{i,j}^{r}h_{j}^{(t-1)}  \right ). $$

The reset and update gates can individually ignore parts of the state vector. The update gates act as conditional leaky integrators that can gate any dimension, either copying it, forgetting it completely, or choosing something between these extremes. There are other variants of the GRU, as this is currently an active area of research. 

## Optimization for Long-Term Dependencies

As was mentioned earlier, long-term dependencies are hard to learn due to the problem of exploding and disappearing gradients over large numbers of time steps. Gated units such as the LSTM and other GRUs are one response to this problem, as these units selectively remember only what they determine most important from each time step. This next section discusses a few of the other methods proposed for dealing with this problem. 

### Clipping Gradients

With highly nonlinear functions, the gradient sometimes includes "cliffs": wide and flat regions sparated by tiny regions where the objective function changes quickly. These cliffs are a challenge for the learning process, since a small weight update can inadvertantly send the weights up the cliff and undo much of the learning that has already taken place. 

The solution to this is to have a smaller learning rate, but it is obviously not desirable to have a tiny learning rate at all times. A more reasonable solution is to clip the gradient.

The primary approach to doing this is to clip the parameter gradient from a minibatch element-wise, just before the parameter update. Another way is to clip the norm $||g||$ of the gradient $g$ before the parameter update:

$$ \text{if } || \textbf{g} || > v $$

$$ g \leftarrow \frac{\textbf{g}v}{||\textbf{g}||}, $$

where $v$ is the norm threshold and $\textbf{g}$ is used to update the parameters. Here the step for each  parameter is still in the direction of the gradient, because the gradient is normalized jointly with a single scaling parameter. This method genearlly keeps gradients from moving up cliffs and exploding. 

### Regularizing to Encourage Information Flow

While gradient clipping helps with exploding gradients, it does not do anything to prevent disappearing ones. One approach to avoiding disappearing gradients was including skip connections so that information flows directly from one time step to another in the intermediate to far future. Another approach was self-loops such as those seen in LSTM cells, which prevent the model from overwriting some of the recurrent unit's previously-learned information. A third approach is to regularize or constrain the parameters so as to encourage "information flow." In particular, we would like the gradient vector $\triangledown_{h^{(t)}}L$ to be back-propogated to maintain its magnitude, even if the loss function only penalizes the output at the end of the sequence. 

Formally, we want: 

$$ \left (  \triangledown_{h^{(t)}}L \right )\frac{\partial h^{(t)}}{\partial h^{(t-1)}}  $$

to be as large as 

$$ \triangledown_{h^{(t)}}L. $$

To achieve this, the follwoing regularizer is used: 

$$ \Omega = \sum_{j} \left (  \frac{ \left || \left (  \triangledown_{h^{(t)}}L \right )\frac{\partial h^{(t)}}{\partial h^{(t-1)}} \right || }{|| \triangledown_{h^{(t)}}L ||}  -1      \right )^{2}. $$

As the gradient of this regularizer is complex to compute, an approximation is used. This regularizer, paired with gradient clipping, can significantly increase the span of dependencies that an RNN can learn. 